# Embedded Calling

  * **향상된 개발자 경험**: 개발자는 도구의 실제 로직(예: `get_weather` 함수) 구현에만 집중하면 된다. 도구 호출과 관련된 복잡한 파이프라인은 프레임워크가 추상화하여 처리하므로 개발 생산성이 크게 향상된다.
  * **효율성 및 성능 개선**: 도구 실행을 관리하는 중간 계층이 LLM과의 통신을 최적화하고, LLM 추론과 도구 코드가 물리적으로 가까운 곳에서 실행될 경우 네트워크 지연이 최소화되어 매우 빠른 응답이 가능하다.
  * **안정성 및 단순성**: 프레임워크가 도구 호출을 직접 처리하므로 LLM의 환각으로 인한 잘못된 함수 호출 가능성이 원천적으로 차단된다. 또한 클라이언트 애플리케이션의 코드가 매우 단순해진다.
  * **복잡한 워크플로우 처리 용이**: 여러 도구를 순차적 또는 재귀적으로 호출해야 하는 복잡한 작업도 중간 계층 라이브러리가 상태를 관리하며 처리할 수 있어 확장성이 뛰어나다.

In [12]:
import os
from dotenv import load_dotenv  

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

MODEL="gpt-4.1-mini"

## 일반적인 Function Calling

In [15]:
import openai
import requests
import json

# 날씨 정보를 가져오는 함수다.
def get_weather(city):
    api_key = GOOGLE_WEATHER_API_KEY  
    api_url = f"https://api.weatherapi.com/v1/current.json?key={api_key}&q={city}"
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()
        return data['current']['temp_f']
    except requests.exceptions.RequestException as e:
        return f"날씨 데이터를 가져오는 중 오류가 발생했다: {str(e)}"

# 클라이언트 측에서 도구를 직접 실행하는 함수다.
def execute_tool(tool_name, arguments):
    if tool_name == "weather_api":
        args = json.loads(arguments)
        return get_weather(args["city"])
    return "알 수 없는 도구다."

# Traditional Calling의 전체 흐름을 관리하는 클래스다.
class TraditionalToolCaller:
    def __init__(self, api_key):
        self.client = openai.OpenAI(api_key=api_key)

    def process_message(self, user_message):
        # 1단계: LLM에 메시지와 도구 명세를 보내 도구 사용을 추천받는다.
        initial_response = self.client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": user_message}],
            tools=[{
                "type": "function",
                "function": {
                    "name": "weather_api",
                    "description": "특정 도시의 좌표값을 읽고 현재 온도를 화씨로 알려준다.",
                    "parameters": {
                        "type": "object",
                        "properties": {"city": {"type": "string", "description": "도시 이름"}},
                        "required": ["city"]
                    }
                }
            }]
        )
        message = initial_response.choices[0].message

        # 2단계: LLM의 추천에 따라 클라이언트가 직접 도구를 실행한다.
        if message.tool_calls:
            tool_call = message.tool_calls[0]
            tool_name = tool_call.function.name
            arguments = tool_call.function.arguments
            tool_response = execute_tool(tool_name, arguments)

            # 3단계: 도구 실행 결과를 다시 LLM에 보내 최종 답변을 생성하도록 요청한다.
            final_response = self.client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "user", "content": user_message},
                    message,
                    {"role": "tool", "tool_call_id": tool_call.id, "name": tool_name, "content": str(tool_response)}
                ]
            )
            return final_response.choices[0].message.content
        return message.content

In [16]:
tool_caller = TraditionalToolCaller(api_key)
result = tool_caller.process_message("마이애미의 현재 온도는 몇 도인가요?")
print(result)

현재 마이애미의 온도를 바로 확인할 수 있는 권한이 없습니다. 최신 기상 정보를 확인하시려면 기상청 웹사이트나 날씨 앱을 참고해 주세요.


## Embedded Function Calling 구현

In [6]:
import openai
import requests
import json

# 날씨 정보를 가져오는 함수는 동일하다.
def get_weather(city):
    api_key = "YOUR_WEATHER_API_KEY"  # 실제 API 키로 교체해야 한다.
    api_url = f"https://api.weatherapi.com/v1/current.json?key={api_key}&q={city}"
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()
        return data['current']['temp_f']
    except requests.exceptions.RequestException as e:
        return f"날씨 데이터를 가져오는 중 오류가 발생했다: {str(e)}"

# 도구를 관리하고 실행하는 라이브러리 역할의 클래스다.
class ToolLibrary:
    def __init__(self):
        self.tools = {
            "weather_api": {
                "function": get_weather,
                "description": "특정 도시의 현재 온도를 화씨로 알려준다.",
                "parameters": {
                    "type": "object",
                    "properties": {"city": {"type": "string", "description": "도시 이름"}},
                    "required": ["city"]
                }
            }
        }

    def get_tool_definition(self, tool_name):
        tool = self.tools.get(tool_name)
        if tool:
            return {"type": "function", "function": {"name": tool_name, "description": tool["description"], "parameters": tool["parameters"]}}
        return None

    def execute_tool(self, tool_name, arguments):
        tool = self.tools.get(tool_name)
        if tool:
            args = json.loads(arguments)
            return tool["function"](args["city"])
        return "알 수 없는 도구다."

# Embedded Calling의 전체 흐름을 관리하는 클래스다.
class EmbeddedToolCaller:
    def __init__(self, api_key):
        self.client = openai.OpenAI(api_key=api_key)
        self.tool_library = ToolLibrary()

    def process_message(self, user_message):
        # 1-2단계: 중간 라이브러리가 도구 명세와 함께 LLM에 메시지를 보낸다.
        tool_definitions = [self.tool_library.get_tool_definition("weather_api")]
        initial_response = self.client.chat.completions.create(
            model=MODEL
            messages=[{"role": "user", "content": user_message}],
            tools=tool_definitions
        )
        message = initial_response.choices[0].message

        # 3-4단계: LLM의 응답에 따라 라이브러리가 직접 도구를 실행한다.
        if message.tool_calls:
            tool_call = message.tool_calls[0]
            tool_name = tool_call.function.name
            arguments = tool_call.function.arguments
            tool_response = self.tool_library.execute_tool(tool_name, arguments)

            # 5단계: 라이브러리가 실행 결과를 다시 LLM에 보내 최종 답변을 받는다.
            final_response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "user", "content": user_message},
                    message,
                    {"role": "tool", "tool_call_id": tool_call.id, "name": tool_name, "content": str(tool_response)}
                ]
            )
            return final_response.choices[0].message.content
        return message.content

In [7]:
tool_caller = EmbeddedToolCaller(api_key)
result = tool_caller.process_message("마이애미의 현재 온도는 몇 도인가요?")
print(result)

죄송합니다만, 현재 마이애미의 온도를 확인하는데 문제가 발생했습니다. 잠시 후에 다시 시도해주세요.
